**Библиотеки**

In [33]:
!pip install transformers

In [34]:
import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
%matplotlib inline

**Данные**

In [35]:
!wget 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
!tar -xf 'aclImdb_v1.tar.gz'


--2021-11-12 11:24:47--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz.1’

aclImdb_v1.tar.gz.1 100%[===================>]  80.23M  48.2MB/s    in 1.7s    

2021-11-12 11:24:49 (48.2 MB/s) - ‘aclImdb_v1.tar.gz.1’ saved [84125825/84125825]



In [36]:
from transformers import BertTokenizer
import torch

In [37]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [38]:
from pathlib import Path

In [39]:
def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')

In [40]:
from sklearn.model_selection import train_test_split

In [41]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.9)

**Модель**

In [42]:
class IMDbDataset(Dataset):
    def __init__(self, texts, labels):
        encoding = tokenizer(texts, return_tensors='pt',
                             padding=True, truncation=True)
        self.input_ids = encoding['input_ids']
        self.attention_mask = encoding['attention_mask']
        # self.labels = lables
        self.labels = torch.tensor(labels).unsqueeze(0)

    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_mask[idx], self.labels[idx]

In [43]:
train_dataset = IMDbDataset(train_texts, train_labels)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [44]:
from transformers import BertForSequenceClassification, AdamW

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', return_dict=True)
model.train()

In [46]:
optimizer = AdamW(model.parameters(), lr=2e-3)

In [ ]:
torch.manual_seed(101)

for param in model.base_model.parameters():
    param.requires_grad = False

n_epochs = 10
train_len = len(train_loader)

batch_losses = []
epoch_losses = []
lb = 0
for epoch in range(1, n_epochs+1):
    batch_loss = 0
    epoch_loss = 0
    for step, (input_ids, attention_mask, labels) in enumerate(train_loader):
        model.zero_grad()
        
        labels = labels.unsqueeze(0)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        epoch_losses.append(outputs.loss)

        lb += outputs.loss
        if not step // 10:
            print('loss:', lb / 10)
            lb = 0

        loss.backward()
        optimizer.step()